In [65]:
import os

from operator import itemgetter

from langchain_openai import ChatOpenAI

from langchain_core.runnables import RunnablePassthrough
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, trim_messages
from langchain_core.chat_history import BaseChatMessageHistory, InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [11]:
llm = ChatOpenAI(model='gpt-4o-mini')

In [21]:
llm.invoke([HumanMessage(content="Hola, soy Jose!")])

AIMessage(content='¡Hola, Jose! ¿Cómo puedo ayudarte hoy?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 12, 'total_tokens': 23}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f33667828e', 'finish_reason': 'stop', 'logprobs': None}, id='run-adb5cfcb-f0a1-4aaa-8b95-d0ba761131d9-0', usage_metadata={'input_tokens': 12, 'output_tokens': 11, 'total_tokens': 23})

In [22]:
llm.invoke([HumanMessage(content="Cuál es mi nombre?")])

AIMessage(content='Lo siento, pero no tengo la capacidad de saber tu nombre a menos que me lo digas. ¿Te gustaría compartirlo?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 12, 'total_tokens': 38}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f33667828e', 'finish_reason': 'stop', 'logprobs': None}, id='run-2008742f-623e-466e-a93b-f2e5fd2ab8af-0', usage_metadata={'input_tokens': 12, 'output_tokens': 26, 'total_tokens': 38})

In [23]:
llm.invoke([
    HumanMessage(content="Hola, soy Jose!"),
    AIMessage(content="Hola Jose!, en qué puedo ayudarte hoy?"),
    HumanMessage(content="Cuál es mi nombre?")
])

AIMessage(content='Tu nombre es Jose. ¿Hay algo más en lo que te gustaría que te ayude?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 34, 'total_tokens': 53}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_5bd87c427a', 'finish_reason': 'stop', 'logprobs': None}, id='run-78ff91db-5114-4b32-a9e0-cafa08482645-0', usage_metadata={'input_tokens': 34, 'output_tokens': 19, 'total_tokens': 53})

In [25]:
store = {}

In [26]:
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

In [28]:
with_message_history = RunnableWithMessageHistory(llm, get_session_history)

In [32]:
config = {"configurable": {"session_id": "abc2"}}

In [33]:
response = with_message_history.invoke(
    [HumanMessage(content="Hola, soy Jose")],
    config=config
)

In [34]:
response.content

'¡Hola, Jose! ¿Cómo puedo ayudarte hoy?'

In [40]:
response = with_message_history.invoke(
    [HumanMessage(content="Cuál es mi nombre?")],
    config=config,
)

In [41]:
for message in store['abc2'].messages:
    print(f'- {message.content}')

- Hola, soy Jose
- ¡Hola, Jose! ¿Cómo puedo ayudarte hoy?
- Cuál es mi nombre?
- Tu nombre es Jose. ¿Hay algo más en lo que te gustaría que te ayudara?


In [43]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. Answer all the questions to the best of your hability. Finish every answer with the word POKEMON"),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [45]:
chain = prompt | llm

In [46]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

In [47]:
config = {"configurable": {"session_id": "abc5"}}

In [48]:
response = with_message_history.invoke(
    [HumanMessage(content="Hola, me llamo Alfredo")],
    config=config
)

In [49]:
response.content

'¡Hola, Alfredo! ¿Cómo puedo ayudarte hoy? POKEMON'

In [50]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | llm

In [51]:
response = chain.invoke(
    {"messages": [HumanMessage(content="hi! I'm bob")], "language": "Japanese"}
)

In [52]:
response.content

'こんにちは、ボブさん！どのようにお手伝いできますか？'

In [53]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history, input_messages_key="messages")

In [54]:
config = {"configurable": {"session_id": "abc11"}}

In [55]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="Hi, I'm Jose!")], "language": "Italian"},
    config=config,
)

In [56]:
response.content

'Ciao, Jose! Come posso aiutarti oggi?'

In [57]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "French"},
    config=config,
)

In [58]:
response.content

"Votre nom est Jose. Comment puis-je vous aider aujourd'hui, Jose ?"

In [61]:
trimmer = trim_messages(
    max_tokens=65,
    strategy="last",
    token_counter=llm,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

In [62]:
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

In [63]:
trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant"),
 HumanMessage(content='whats 2 + 2'),
 AIMessage(content='4'),
 HumanMessage(content='thanks'),
 AIMessage(content='no problem!'),
 HumanMessage(content='having fun?'),
 AIMessage(content='yes!')]

In [67]:
chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer)
    | prompt
    | llm
)

In [68]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what's my name?")],
        "language": "English",
    }
)

In [69]:
response.content

"I'm sorry, but I don't know your name. If you'd like to share it, feel free!"

In [70]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)

In [71]:
response.content

'You asked for the sum of 2 + 2.'

In [ ]:
trimmer = trim_messages(
    max_tokens=65,
    strategy="last",
    token_counter=llm,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

In [ ]:
chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer)
    | prompt
    | llm
)

In [72]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

In [ ]:
config = {"configurable": {"session_id": "abc20"}}

In [73]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

In [74]:
response.content

'I don’t know your name. How would you like me to address you?'

In [75]:
config = {"configurable": {"session_id": "abc15"}}
for r in with_message_history.stream(
    {
        "messages": [HumanMessage(content="hi! I'm todd. tell me a joke")],
        "language": "English",
    },
    config=config,
):
    print(r.content, end="|")

|Hi| Todd|!| Here|’s| a| joke| for| you|:

|Why| did| the| scare|crow| win| an| award|?

|Because| he| was| outstanding| in| his| field|!||